<center><img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective">Objective</h3>
<br><strong style="color: #000000;"><em>Find and Delete Redshift Clusters with Low CPU Utilization</em></strong></div>
</center>
<p>&nbsp;</p>
<center>
<h2 id="Delete-RDS-Instances-with-Low-CPU-Utilization"><u>Delete Redshift Clusters with Low CPU Utilization</u></h2>
</center>
<h1 id="Steps-Overview">Steps Overview</h1>
<p>1)<a href="#1" target="_self" rel="noopener"> Find Redshift Clusters with Low CPU Utilization</a><br>2)<a href="#2" target="_self" rel="noopener"> Delete Redshift Cluster</a></p>

In [4]:
if region == None:
    region = ''
if cluster_identifiers and not region:
    raise SystemExit("Please provide a region for the Redshift Clusters names!")

<h3 id="Find-RDS-Instances-with-Low-CPU-Utilization"><a id="1" target="_self" rel="nofollow"></a>Find Redshift Clusters with Low CPU Utilization</h3>
<p>Using unSkript's Find Redshift Clusters with Low CPU Utilization action, we will find instances with a low CPU utilization given a threshold percentage using the&nbsp;<span style="color: rgb(53, 152, 219);">CPUUtilization <span style="color: rgb(0, 0, 0);">attribue found in the statistics list of&nbsp; <em>get_metric_statistics</em> API of Cloudwatch.</span></span></p>
<blockquote>
<p>This action takes the following parameters: <code>region, threshold, duration_minutes</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>low_cpu_clusters</code></p>
</blockquote>

In [ ]:
##
##  Copyright (c) 2023 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Optional, Tuple
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
import pprint
from datetime import datetime,timedelta


from beartype import beartype
@beartype
def aws_find_redshift_clusters_with_low_cpu_utilization_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_find_redshift_clusters_with_low_cpu_utilization(handle, utilization_threshold:int=10, region: str = "", duration_minutes:int=5) -> Tuple:
    """aws_find_redshift_clusters_with_low_cpu_utilization finds Redshift Clusters that have a lower cpu utlization than the given threshold

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type region: string
        :param region: Region of the Cluster.

        :type utilization_threshold: integer
        :param utilization_threshold: The threshold percentage of CPU utilization for a Redshift Cluster.

        :type duration_minutes: integer
        :param duration_minutes: The threshold percentage of CPU utilization for a Redshift Cluster.

        :rtype: status, list of clusters and their region.
    """
    result = []
    all_regions = [region]
    if not region:
        all_regions = aws_list_all_regions(handle)
    for reg in all_regions:
        try:
            redshiftClient = handle.client('redshift', region_name=reg)
            cloudwatchClient = handle.client('cloudwatch', region_name=reg)
            for cluster in redshiftClient.describe_clusters()['Clusters']:
                cluster_identifier = cluster['ClusterIdentifier']
                response = cloudwatchClient.get_metric_statistics(
                Namespace='AWS/Redshift',
                MetricName='CPUUtilization',
                Dimensions=[
                    {
                        'Name': 'ClusterIdentifier',
                        'Value': cluster_identifier
                    }
                ],
                StartTime=(datetime.utcnow() - timedelta(minutes=duration_minutes)).isoformat(),
                EndTime=datetime.utcnow().isoformat(),
                Period=60,
                Statistics=['Average']
                )
                if len(response['Datapoints']) != 0:
                    cpu_usage_percent = response['Datapoints'][-1]['Average']
                    if cpu_usage_percent < utilization_threshold:
                        cluster_dict = {}
                        cluster_dict["region"] = reg
                        cluster_dict["cluster"] = cluster_identifier
                        result.append(cluster_dict)
        except Exception:
            pass

    if len(result) != 0:
        return (False, result)
    else:
        return (True, None)




task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "duration_minutes": "int(duration_minutes)",
    "utilization_threshold": "int(threshold)",
    "region": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "region",
    "iter_parameter": "region"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not cluster_identifiers",
    "condition_result": true
    }''')
task.configure(outputName="low_cpu_clusters")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_find_redshift_clusters_with_low_cpu_utilization, lego_printer=aws_find_redshift_clusters_with_low_cpu_utilization_printer, hdl=hdl, args=args)

<h3 id="Create-List-of-Low-CPU-Utilization-RDS-Instances">Create List of Low CPU Utilization Redshift Clusters</h3>
<p>This action filters regions that have no clusters with low CPU utilization and creates a list of those that have them.</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>all_low_cpu_clusters</code></p>
</blockquote>

In [ ]:
all_low_cpu_clusters = []
try:
    for res in low_cpu_clusters:
        if type(res)==bool:
            if res == False:
                continue
        elif type(res)==list:
            if len(res)!=0:
                all_low_cpu_clusters=res
except Exception:
    for ins_identifier in cluster_identifiers:
        data_dict = {}
        data_dict["region"] = region
        data_dict["cluster"] = ins_identifier
        all_low_cpu_clusters.append(data_dict)
print(all_low_cpu_clusters)
task.configure(outputName="all_low_cpu_instances")

<h3 id="Delete-RDS-Instance"><a id="2" target="_self" rel="nofollow"></a>Delete Redshift Clusters</h3>
<p>This action deletes instances found in Step 1.&nbsp; By default, the skip final cluster screenshot is set to <span style="color: rgb(224, 62, 45);">False.&nbsp;<span style="color: rgb(0, 0, 0);">This setting will not take a final screenshot of the cluster.</span></span></p>
<blockquote>
<p>This action takes the following parameters:&nbsp;<code>cluster, region, skip_final_cluster_screenshot</code></p>
</blockquote>

In [ ]:
##
# Copyright (c) 2023 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Optional, Dict
import pprint


from beartype import beartype
@beartype
def aws_delete_redshift_cluster_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_delete_redshift_cluster(handle, region: str, cluster_identifier: str, skip_final_cluster_snapshot:bool=False) -> Dict:
    """aws_delete_redshift_cluster dict response.

        :type region: string
        :param region: AWS Region.

        :type cluster_identifier: string
        :param cluster_identifier: The identifier of the cluster to be deleted.

        :type skip_final_cluster_snapshot: boolean
        :param skip_final_cluster_snapshot: Determines whether a final snapshot of the cluster is created before Amazon Redshift deletes the cluster. If true, a final cluster snapshot is not created. If false, a final cluster snapshot is created before the cluster is deleted.

        :rtype: dict of response
    """
    try:
        redshiftClient = handle.client('redshift', region_name=region)
        response = redshiftClient.delete_cluster(
            ClusterIdentifier=cluster_identifier,
            SkipFinalClusterSnapshot=skip_final_cluster_snapshot
            )
        return response
    except Exception as e:
        raise Exception(e)




task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "skip_final_cluster_snapshot": "False",
    "cluster_identifier": "iter.get(\\"cluster\\")",
    "region": "iter.get(\\"region\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "all_low_cpu_clusters",
    "iter_parameter": ["cluster_identifier","region"]
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(all_low_cpu_clusters)!=0",
    "condition_result": true
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_delete_redshift_cluster, lego_printer=aws_delete_redshift_cluster_printer, hdl=hdl, args=args)

<h3 id="Conclusion&para;">Conclusion</h3>
<p>In this Runbook, we were able to filter low CPU utilization Redshift Clusters given threshold percentage and delete them. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>